# PDFドキュメントの読込

# 準備

In [6]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
import openai

# 環境変数の読み込み
load_dotenv()
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']
openai.api_key = os.environ['API_KEY'] # これを追記しないとエラーになる

# Indexの構築

In [7]:
from pathlib import Path
from llama_index import download_loader

# PDFReaderの取得
PDFReader = download_loader("PDFReader")

# PDFドキュメントからテキスト情報を読込
loader = PDFReader()
documents = loader.load_data(file=Path('./data2/001615363.pdf'))

In [9]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.constants import DEFAULT_CHUNK_OVERLAP, DEFAULT_CHUNK_SIZE
import tiktoken
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index import VectorStoreIndex

# テキスト分割の指定
text_splitter = TokenTextSplitter(
    separator="。",
    chunk_size=DEFAULT_CHUNK_SIZE,
    chunk_overlap=DEFAULT_CHUNK_OVERLAP,
    tokenizer=tiktoken.get_encoding("cl100k_base").encode)

# NodeParserの作成
node_parser = SimpleNodeParser(text_splitter=text_splitter)

# 言語モデルの指定
llm = OpenAI(model='gpt-3.5-turbo', temperature=1.2)

# サービスコンテキストの作成
service_context = ServiceContext.from_defaults(llm=llm, node_parser=node_parser)

# Indexの構築
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [38]:
# 参考：言語モデルのエンコーディングを調べる
import tiktoken
print(tiktoken.encoding_for_model('gpt-3.5-turbo'))

<Encoding 'cl100k_base'>


# Query Engineの作成

In [33]:
from llama_index import QuestionAnswerPrompt

# テンプレートのカスタマイズ
QA_TEMPLATE= (
    "コンテキスト情報は以下のとおりです。 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "事前知識ではなくコンテキスト情報が与えられた場合, "
    "日本語で質問に答えてください: {query_str}\n"
)

# Query Engineの作成
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=3,
    streaming=True,
    text_qa_template=QuestionAnswerPrompt(QA_TEMPLATE),
)

# ユーザーからの質問に回答

In [35]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
response.print_response_stream()

訪日外国人のお土産の購入状況は、以下のとおりです。

1. 菓子類を購入した人の割合は70.4％です。 
2. その他に、食料品・飲料・たばこ（43.4％）と衣類（33.8％）も購入されています。 
3. 国籍別では、韓国、台湾、香港、中国の訪日外国人が特に菓子類を多く購入しています。 
4. 宝石・貴金属の購入者単価は最も高く、8.3万円です。台湾と香港の訪日外国人が比較的高額な購入をしています。

In [36]:
# 出典を表示
print(response.get_formatted_sources())

> Source (Node id: b585c7e0-df7a-481e-8eb9-d9422d411c34): 訪日外国人消費動向調査  
 
20 ３．土産品の購入実態  
 
（１） 費目別購入率 
 費目別の 購入率（その費目を購入した人の
割合）は「菓子類」 （ 70.4％）、「その他食
料品・...

> Source (Node id: 84e4e253-3a8c-428b-abc6-9b4cd9db3b19): 訪日外国人消費動向調査 。 。38 。 （３）買物場所 。百貨店・デパート 。原則として百貨店協会加盟の店舗 。家電量販店 。PCやカメラ、電気製品を専門に販売する店舗 。ファッション専門店 。...

> Source (Node id: 88d8dfca-437b-439a-ad3e-bb4da7019dcb): 。  図表3-1 買物代の費目別購入率および購入者単価  （主要国籍･地域別 、全目的） 。図表3-2 買物代の費目別購入率および 購入者単価  （主要国籍･地域別 、観光・レジャー目的 ） 。...


# 詳細なログの表示

In [25]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [26]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u304a\\u571f\\u7523\\u306e\\u8cfc\\u5165\\u72b6\\u6cc1\\u3092\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
api_version=None data='{"input": ["\\u304a\\u571f\\u7523\\u306e\\u8cfc\\u5165\\u72b6\\u6cc1\\u3092\\u6559\\u3048\\u3066"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=133 request_id=c9963cf7bb9e7e9395555e381f901c7c response_code=200
message='OpenAI API response' path=https://api.opena

In [29]:
# もとに戻す（ログを出力しない）
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
# ログハンドラー出力を除去
while len(logging.getLogger().handlers) > 0:
  logging.getLogger().removeHandler(logging.getLogger().handlers[0])

# ストレージに保存

In [27]:
# ストレージに保存
index.storage_context.persist("./storage02")

DEBUG:fsspec.local:open file: c:/sample/storage02/docstore.json
open file: c:/sample/storage02/docstore.json
DEBUG:fsspec.local:open file: c:/sample/storage02/index_store.json
open file: c:/sample/storage02/index_store.json
DEBUG:fsspec.local:open file: c:/sample/storage02/vector_store.json
open file: c:/sample/storage02/vector_store.json
DEBUG:fsspec.local:open file: c:/sample/storage02/graph_store.json
open file: c:/sample/storage02/graph_store.json


In [28]:
from llama_index import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage02")
# Indexのロード
index = load_index_from_storage(storage_context)

DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage02\docstore.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage02\docstore.json.
DEBUG:fsspec.local:open file: c:/sample/storage02/docstore.json
open file: c:/sample/storage02/docstore.json
DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage02\index_store.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage02\index_store.json.
DEBUG:fsspec.local:open file: c:/sample/storage02/index_store.json
open file: c:/sample/storage02/index_store.json
DEBUG:llama_index.vector_stores.simple:Loading llama_index.vector_stores.simple from ./storage02\vector_store.json.
Loading llama_index.vector_stores.simple from ./storage02\vector_store.json.
DEBUG:fsspec.local:open file: c:/sample/storage02/vector_store.json
open file: c:/sample/storage02/vector_store.json
DEBUG:llama_index.graph_stores

In [30]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

コンテキスト情報によれば、訪日外国人のお土産の購入状況について述べられています。図表3-1によれば、外国人観光客の購入率が高い費目は「菓子類」（70.4%）、次に「その他食料品・飲料・たばこ」（43.4%）、そして「衣類」（33.8%）です。国籍・地域別で最も高い購入率が認められるのは、韓国、台湾、香港、中国の外国人客が「菓子類」を主に購入しており（それぞれ82.9%、72.9%、63.3%、73.1%）、米国の外国人客は「その他食料品・飲料・たばこ」を主に購入しています（54.1%）。また、購入者単価（1人当たりの平均支出）については、「宝石・貴金属」が最も高く（8.3万円）、台湾と香港の外国人客は他の国籍・地域に比べて「宝石・貴金属」をより高い金額で購入しています（それぞれ19.9万円、16.6万円）。
